In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,DateType

In [0]:
driver_schema = StructType([
    StructField('driverId', IntegerType(),True),
    StructField('driverRef', StringType(),True),
    StructField('number', IntegerType(),True),
    StructField('code', StringType(),True),
    StructField('name', StructType([StructField('forename', StringType(), True), StructField('surname', StringType(), True)
    ]), True),
    StructField('dob', DateType(),True),
    StructField('nationality', StringType(), True),
    StructField('url', StringType(),True)
])

In [0]:
df = spark.read.json('abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/RAW-V1/drivers.json', schema=driver_schema)

In [0]:
df.display()